In [ ]:
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import scipy as sp
import urllib.parse
import re

We can see two base url used to retrive data, the ```URL_FORM_BASE``` for the form and ```URL_DATA_BASE``` for the data (the table which contains the actual information).

In [ ]:
URL_FORM_BASE = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter"
URL_DATA_BASE = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html"

By first inspecting the source code of the page we can see that the form has some hidden field, they indicate to the backend what kind of data to retrieve. We can thus form a dictionary of "default parameters".

In [ ]:
FORM_PARAM_DEFAULT = {
    "ww_v_list" : 1,
    "ww_i_reportmodel": "133685247",
    #"ww_c_langue": 'en',
}

We then take interest in the form itself. We can see that we have 5 different fields each with a specific name attribute. (When the form is "posted" the name attribute is used to generate the url). We can thus retrieve all the options for each field relying on his name attribute.

We will only work with the HTML data so we directly take the parameters from the source code and don't bother to parse the HMTL page to retrieve it (for HMTL use a value of 133685270 for the ww_i_reportModelXsl attribute).

In [ ]:
def request2soup(url, params):
    r = requests.get(url, params=params)
    assert r.status_code == 200
    return BeautifulSoup(r.text, "lxml")

def get_select_options(soup, name_attr):
    data = soup.select('select[name={}]'.format(name_attr))[0] #We assume that we have only one field with a specific name
    options = {option['value']: option.text for option in data.find_all('option', value=True, selected=False) if  option.text != ''}
    return options

def get_fields_options(soup, fields):
    form_options = {}
    for key, value in fields.items():
        form_options[value] = get_select_options(soup, value)
    
    return form_options

FORM_FIELDS = {
    # Label : Name Attribute
    'Unité académique': 'ww_x_UNITE_ACAD',
    'Période académique': 'ww_x_PERIODE_ACAD',
    'Période pédagogique': 'ww_x_PERIODE_PEDAGO',
    'Type de semestre': 'ww_x_HIVERETE',
}

soup_index = request2soup(URL_FORM_BASE, FORM_PARAM_DEFAULT)

FORM_FIELDS_OPTIONS = get_fields_options(soup_index, FORM_FIELDS)

In [ ]:
FORM_FIELDS_OPTIONS

By submiting the form, we get a table filled with links at the bottom of the form back. Of course ISA does not want to play it simple and rely instead of the href attribute to javascript for the link behaviour. A clicked link call the js function "loadReport" which generate the "data url" based on the ww_x_GPS attribute. 

First we define two utilities function to generate for us the complete url based on the given parameters.

In [ ]:
# Get Url for the form page (the one with the links)
def get_form_url(params):
    return URL_FORM_BASE + "?" + urllib.parse.urlencode(dict(params, **FORM_PARAM_DEFAULT))

# Get Url for the data page (the one with the table of students information)
def get_data_url(params, GPS=-1, type=133685270):
    p = {'ww_x_GPS': GPS, 'ww_i_reportModelXsl': type}
    return URL_DATA_BASE + "?" + urllib.parse.urlencode(dict(dict(params, **FORM_PARAM_DEFAULT), **p))

Instead of parsing the form page to retrieve the ww_x_GPS attribute, we generate directly the data url based on the form options discovered above.

In [ ]:
# Returns a dataframe containing all the student information from 2007 to 2016 for the given pedagogic period
def get_data(period_peda, columns):
    data_list = []
    for peda in period_peda:
        for period in FORM_FIELDS_OPTIONS['ww_x_PERIODE_ACAD']:
            params = {
                'ww_x_UNITE_ACAD': 249847, # hard coded: only interested in the computer science faculty
                'ww_x_PERIODE_ACAD': period,
                'ww_x_PERIODE_PEDAGO': peda,
                'ww_x_HIVERETE': '',
            }  
            
            try: 
                # By specifying ww_x_UNITE_ACAD, ww_x_PERIODE_ACAD, ww_x_PERIODE_PEDAGO we are sur 
                # that only one table will be present on the page. Indeed there was only one BA1 class in year
                # for the computer science faculty
                data = pd.read_html(get_data_url(params), header=1)[0]
                df = data.copy()[columns] # Only keep interesting columns
                df['semester'] = period_peda[peda] # Add semester information
                df['period'] = FORM_FIELDS_OPTIONS['ww_x_PERIODE_ACAD'][period] # Add period information
                data_list.append(df)
            except:
                # read_html might throw an error if there is no table in the page
                print("No Data : " +  period_peda[peda] + " " + FORM_FIELDS_OPTIONS['ww_x_PERIODE_ACAD'][period])
        
    d = pd.concat(data_list) 
    return d

--------------

# Q1 : Bachelor Data Analysis

In [ ]:
bachelor = {  
    '249108': 'Bachelor semestre 1',
    '249114': 'Bachelor semestre 2',
    '942155': 'Bachelor semestre 3',
    '942163': 'Bachelor semestre 4',
    '942120': 'Bachelor semestre 5',
    '942175': 'Bachelor semestre 6',
    #'2226768': 'Bachelor semestre 5b', # No data
    #'2226785': 'Bachelor semestre 6b', # No data
}
columns = ['No Sciper', 'Statut', 'Civilité']

# Retrieve the data
d = get_data(bachelor, columns)
d['No Sciper'] = d['No Sciper'].apply(str)
bachelor = d.copy()

Now that we have the data in a pandas dataframe we can start working on it.

# TODO

### Assumption
 * how we computed the time elapsed esp. the + .5 in the formula
 * why there are already values for year 2017 (student abroad for 3rd year)
 * How did a guy spent 7 years for his bachelor (one year break)

In [ ]:
# Return if Dataframe has both ba1 ba6 entries
first_and_last = lambda semesters: "Bachelor semestre 1" in semesters and "Bachelor semestre 6" in semesters
# Return if the semester is an autumn semester or not
is_autumn = lambda sem: int(sem[-1:]) % 2 == 1
# Return if we should keed the row (based on the is_autumn result)
idx_to_keep = lambda sem: 0 if sem else 1

In [ ]:
grouped = d.groupby(['No Sciper'])
# Only keep the student for which we have at least information about BA1 and BA6
filtered = grouped.filter(lambda x: len(x.values) >= 2 and first_and_last(x.values)).copy()

In [ ]:
# Add extracted information to ease our analysis
filtered['is_autumn'] = filtered['semester'].apply(lambda x: is_autumn(x))
filtered['date'] = filtered.apply(lambda x: int(x['period'].split("-")[idx_to_keep(x['is_autumn'])]), axis=1)
filtered['date_help'] = filtered.apply(lambda x: float(x.date) + (.5 if x.is_autumn else 0), axis=1)

In [ ]:
grpd = filtered.groupby(['No Sciper'])
duration = grpd.apply(lambda x: max(x.date_help) - min(x.date_help) + .5)
# Finally add total time spend in bachelor
filtered['time_elapsed'] = filtered['No Sciper'].apply(lambda sciper: duration[sciper])

In [ ]:
# Strip down the dataframe to a more usefull representation
df = filtered.groupby(['No Sciper'])['Civilité', 'time_elapsed'].agg(lambda x:x.value_counts().index[0])

#### Duration distribution by sex
# TODO Comments

In [ ]:
tmp = df.reset_index().groupby(['Civilité','time_elapsed']).count()
tmp = tmp.groupby(level=0).apply(lambda x: 100*x/float(x.sum()))
tmp = tmp.unstack(level=0)
graph = tmp.plot(kind='bar')
graph.set_xlabel('Length (in year)')
graph.set_ylabel('Percentage')
graph.set_title('Bachelor length by sex')

#### Duration mean by sex
# TODO Comments & Hypothesis testing
Talk about the fact that the number of women is low thus a single occurence of a high number of semester might skew the whole thing

In [ ]:
df.groupby(['Civilité']).describe()

# Q2 : Master Data Analysis

In [ ]:
master = {  
    #'953137':  'Stage automne 3ème année', # No Data
    #'983606':  'Stage printemps 3ème année', # No Data
    #'2226616': 'Stage automne 4ème année', # No Data
    #'2226626': 'Stage printemps 4ème année', # No Data
    #'2227132': 'Stage printemps master', # No Data
    '2230106': 'Master semestre 1',
    '942192':  'Master semestre 2',
    '2230128': 'Master semestre 3',
    #'2230140': 'Master semestre 4', # No Data
    #'2335667': 'Mineur semestre 1', # No Data
    #'2335676': 'Mineur semestre 2', # No Data
    #'2754553': 'Semestre printemps', # No Data
    #'953159':  'Semestre automne', # No Data
    '249127':  'Projet Master automne',
    '3781783': 'Projet Master printemps',
}


columns = ['No Sciper', 'Statut', 'Civilité', 'Mineur', 'Spécialisation',]

# Retrieve the data
d = get_data(master, columns)
d['No Sciper'] = d['No Sciper'].apply(str)
master = d.copy()

In [ ]:
master.head()

# TODO

### Assumption
 * Assumes that if a student had once a minor, he has to do his master in at least three semester
 * Don't verify that a student that has a minor and a stage do 4 semester with courses
 * If student has neither a entry for PDM or a mention of a stage semester we add 6 months to the duration
 * If student has only a stage semester add 6 months for the PDM
 * If student has only PDM, don't add anything

In [ ]:
# Return if student has finished his master
def first_and_last(rows):
    minor = sum([np.nan in [x[3]] for x in list]) > 0
    spe = sum([np.nan in [x[4]] for x in list]) > 0
    
    if minor or spe:
        return "Master semestre 1" in rows and "Master semestre 3" in rows

    return "Master semestre 1" in rows and "Master semestre 2" in rows
    
# Return if the semester is an autumn semester or not
def is_autumn(sem):
    if 'automne' in sem:
        return True
    elif 'printemps' in sem:
        return False
    else:
        return int(sem[-1:]) % 2 == 1  
    
# Return if we should keed the row (based on the is_autumn result)
idx_to_keep = lambda sem: 0 if sem else 1

def add_semester(rows):
    projet = rows['semester'].str.contains('Projet').max()
    stage = rows['Statut'].str.contains('Stage').max()
    return (not projet and stage) or (not projet and not stage)

def get_spe(rows):
    spe = rows[rows['semester'] == 'Master semestre 3']['Spécialisation'].value_counts()
    if len(spe):
        assert len(spe) == 1
        return spe.index[0]
    return None

def get_minor(rows):
    spe = rows[rows['semester'] == 'Master semestre 3']['Mineur'].value_counts()
    if len(spe):
        assert len(spe) == 1
        return spe.index[0]
    return None

In [ ]:
grouped = d.groupby(['No Sciper'])
# Only keep the student for which we have information about at least MA1 and MA2
filtered = grouped.filter(lambda x: len(x.values) >= 2 and first_and_last(x.values)).copy()

In [ ]:
# Add extracted information to ease our analysis
filtered['is_autumn'] = filtered['semester'].apply(lambda x: is_autumn(x))
filtered['date'] = filtered.apply(lambda x: int(x['period'].split("-")[idx_to_keep(x['is_autumn'])]), axis=1)
filtered['date_help'] = filtered.apply(lambda x: float(x.date) + (.5 if x.is_autumn else 0), axis=1)

In [ ]:
grpd = filtered.groupby(['No Sciper', 'Civilité'])
duration = grpd.apply(lambda x: max(x.date_help) - min(x.date_help) + .5)
add = grpd.apply(lambda x: add_semester(x)).astype('category')
minor = grpd.apply(lambda x: get_minor(x)).astype('category')
spe = grpd.apply(lambda x: get_spe(x)).astype('category')

print(len(add))
df = pd.concat({'duration': duration, 'add_semester':add, 'spe':spe, 'minor':minor }, axis=1)
df.head()